# Notebook for deploying on aws and testing model

BLABLABALBALBA Lorem Ipsum BALBALABLAB

---

In [1]:
# import aws sagemaker apis
import os
import boto3
import sagemaker
from sagemaker import get_execution_role

In [2]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# default S3 bucket
bucket = sagemaker_session.default_bucket()

In [3]:
# specify where to upload in S3
data_dir = './data/dataset/'
prefix = 'dog-breed-data'

s3_keys = [obj.key for obj in boto3.resource('s3').Bucket(bucket).objects.all()]
# upload to S3 - Due to the size of the dataset it takes some time to finish...
if any(prefix in s for s in s3_keys):
    input_data = f"s3://{bucket}/{prefix}"
else:
    input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)

s3://sagemaker-us-east-1-031843945636/dog-breed-data


## PyTorch Estimator

In [4]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
# prefix is specified above
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='source', # this should be just "source" for your code
                    role=role,
                    framework_version='1.5.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'output_dim': 120,
                        'hidden_dim': 256,
                        'epochs': 5, # could change to higher
                        'batch-size': 256
                    })

In [ ]:
estimator.fit({'train': os.path.join(input_data, 'train'), 'test': os.path.join(input_data, 'test')})

2020-05-21 16:44:30 Starting - Starting the training job..